In [2]:
import os
import glob
import datetime
import numpy as np
import pandas as pd

pd.set_option('max_rows', 100)
pd.set_option('max_columns', 100)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

startDate = '20201014'
endDate = '20201014'

sl = ['SZ123015', 'SZ123032', 'SZ128070', 'SZ123042', 'SZ123034',
       'SZ123048', 'SZ123061', 'SH113576', 'SZ123065', 'SZ123030',
       'SZ123045', 'SZ123013', 'SZ127004', 'SZ128103', 'SZ128073',
       'SZ128041', 'SZ123012', 'SZ123062', 'SH110044', 'SZ128079',
       'SZ123052', 'SZ128036', 'SZ123014', 'SZ128091', 'SZ123018',
       'SZ123031', 'SZ123038', 'SH113035', 'SZ128052', 'SZ128053',
       'SZ128039', 'SZ128050', 'SZ123023', 'SH113555', 'SZ127008',
       'SZ123047', 'SZ123051', 'SZ123024', 'SZ123029', 'SZ128076',
       'SZ123026', 'SZ123027', 'SZ128112', 'SH113521', 'SZ123057',
       'SZ128108', 'SH113520', 'SZ123046', 'SH113038', 'SH113011']
sl1 = [i[2:] for i in sl if i[:2] == 'SZ']
sl2 = [i[2:] for i in sl if i[:2] == 'SH']
sl1 = [int(i) + 2000000 for i in sl1]
sl2 = [int(i) + 1000000 for i in sl2]

readPath = r'A:\KR_daily_data\***'
dataPathLs = np.array(glob.glob(readPath))
dateLs = np.array([os.path.basename(i) for i in dataPathLs])
dateLs = dateLs[(dateLs >= startDate) & (dateLs <= endDate)]

for ii in dateLs:
    order = []
    trade = []
    snapshot = []
    os.mkdir('G:\\convertible_bond1\\' + ii)
    y = ii
    
    readPath = '\\\\192.168.10.34\\trading\\dailyRawData\\' + ii + '\\logs_***_zs_96_03_day_pcap\\mdOrderPcap_***.csv'
    dataPathLs = np.array(glob.glob(readPath))
    try:
        orderLog = pd.read_csv(dataPathLs[0])
        print(dataPathLs[0])
    except:
        continue
    
    orderLog = orderLog.rename(columns={"time":'TransactTime', 'ID':'skey'})
    orderLog = orderLog[orderLog['skey'].isin(sl1)]
    orderLog["OrderType"] = np.where(orderLog["OrderType"] == 'U', 3, orderLog["OrderType"])

    orderLog['date'] = int(y)
    orderLog['time1'] = int(y) * 1000000000 + orderLog['TransactTime']
    orderLog["TransactTime"] = orderLog['TransactTime'].astype('int64') * 1000
    orderLog["clockAtArrival"] = orderLog["time1"].astype(str).apply(
        lambda x: np.int64(datetime.datetime.strptime(x, '%Y%m%d%H%M%S%f').timestamp() * 1e6))
    orderLog.drop("time1", axis=1, inplace=True)
    orderLog['datetime'] = orderLog["clockAtArrival"].apply(lambda x: datetime.datetime.fromtimestamp(x / 1e6))
    orderLog = orderLog.rename(columns={"TransactTime":"time"})

    for col in ["skey", "date", "ApplSeqNum", "OrderQty", "Side", "OrderType"]:
        orderLog[col] = orderLog[col].astype('int32')
    for cols in ["Price"]:
        orderLog[cols] = (orderLog[cols]/10000).round(3)

    assert(orderLog[((orderLog["Side"] != 1) & (orderLog["Side"] != 2)) | (orderLog["OrderType"].isnull())].shape[0] == 0)

    orderLog = orderLog.rename(columns={"Side":"order_side", "OrderType":"order_type", "Price":"order_price", "OrderQty":"order_qty"})
    orderLog = orderLog[["skey", "date", "time", "clockAtArrival", "datetime", "ApplSeqNum", "order_side", "order_type", "order_price",
                                                 "order_qty"]]
    


    readPath = '\\\\192.168.10.34\\trading\\dailyRawData\\' + ii + '\\logs_***_zs_96_03_day_pcap\\mdTradePcap_SZ_***.csv'
    dataPathLs = np.array(glob.glob(readPath))
    try:
        tradeLog = pd.read_csv(dataPathLs[0])
        print(dataPathLs[0])
    except:
        continue
    
    tradeLog = tradeLog.rename(columns={"time":'TransactTime', 'ID':'skey'})
    tradeLog["TradeBSFlag"] = 'N'
    tradeLog = tradeLog[tradeLog['skey'].isin(sl1)]
    tradeLog['date'] = int(y)
    tradeLog['time1'] = int(y) * 1000000000 + tradeLog['TransactTime']
    tradeLog["TransactTime"] = tradeLog['TransactTime'].astype('int64') * 1000
    tradeLog["clockAtArrival"] = tradeLog["time1"].astype(str).apply(
        lambda x: np.int64(datetime.datetime.strptime(x, '%Y%m%d%H%M%S%f').timestamp() * 1e6))
    tradeLog.drop("time1", axis=1, inplace=True)
    tradeLog['datetime'] = tradeLog["clockAtArrival"].apply(lambda x: datetime.datetime.fromtimestamp(x / 1e6))
    tradeLog['TradePrice'] = (tradeLog['TradePrice'] / 10000).round(3)
    tradeLog = tradeLog.rename(columns={"TradeQty":"trade_qty", "TradePrice":"trade_price", "ExecType":"trade_type", 'TransactTime':'time'})
    tradeLog["trade_flag"] = 0
    tradeLog["trade_type"] = np.where(tradeLog["trade_type"] == 'F', 1, tradeLog["trade_type"])
    for col in ["skey", "date", "ApplSeqNum", "BidApplSeqNum", "OfferApplSeqNum", "trade_qty", "trade_type", "trade_flag"]:
        tradeLog[col] = tradeLog[col].astype('int32')

    tradeLog = tradeLog[["skey", "date", "time", "clockAtArrival", "datetime", "ApplSeqNum", "trade_type", "trade_flag",
                                                 "trade_price", "trade_qty", "BidApplSeqNum", "OfferApplSeqNum"]]
    


    readPath = '\\\\192.168.10.34\\trading\\dailyRawData\\' + ii + '\\logs_***_zs_96_03_day_pcap\\mdL2Pcap_SZ_***.csv'
    dataPathLs = np.array(glob.glob(readPath))
    try:
        SZ = pd.read_csv(dataPathLs[0])
        print(dataPathLs[0])
    except:
        continue
    SZ = SZ.loc[:, ["sequenceNo", "ID", "time", "cum_volume", "cum_amount", "cum_tradesCnt", "prevClose",
                                "open", "high", "low", "close", 'bid10p','bid9p','bid8p','bid7p','bid6p','bid5p','bid4p','bid3p','bid2p','bid1p',
                                'ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p', 'bid10q','bid9q','bid8q',
                                  'bid7q','bid6q','bid5q','bid4q','bid3q','bid2q','bid1q', 'ask1q','ask2q','ask3q','ask4q','ask5q','ask6q',
                                  'ask7q','ask8q','ask9q','ask10q', 'bid10n', 'bid9n', 'bid8n', 'bid7n', 'bid6n', 'bid5n', 'bid4n', 'bid3n', 'bid2n', 'bid1n', 
                                  'ask1n', 'ask2n', 'ask3n', 'ask4n', 'ask5n', 'ask6n','ask7n', 'ask8n', 'ask9n', 'ask10n','bid1Top1q','bid1Top2q','bid1Top3q','bid1Top4q','bid1Top5q','bid1Top6q',
            'bid1Top7q','bid1Top8q','bid1Top9q','bid1Top10q','bid1Top11q','bid1Top12q','bid1Top13q','bid1Top14q','bid1Top15q','bid1Top16q','bid1Top17q','bid1Top18q',
            'bid1Top19q','bid1Top20q','bid1Top21q','bid1Top22q','bid1Top23q','bid1Top24q','bid1Top25q','bid1Top26q','bid1Top27q','bid1Top28q','bid1Top29q',
            'bid1Top30q','bid1Top31q','bid1Top32q','bid1Top33q','bid1Top34q','bid1Top35q','bid1Top36q','bid1Top37q','bid1Top38q','bid1Top39q','bid1Top40q',
            'bid1Top41q','bid1Top42q','bid1Top43q','bid1Top44q','bid1Top45q','bid1Top46q','bid1Top47q','bid1Top48q','bid1Top49q','bid1Top50q', 'ask1Top1q',
            'ask1Top2q','ask1Top3q','ask1Top4q','ask1Top5q','ask1Top6q','ask1Top7q','ask1Top8q','ask1Top9q','ask1Top10q','ask1Top11q','ask1Top12q','ask1Top13q',
            'ask1Top14q','ask1Top15q','ask1Top16q','ask1Top17q','ask1Top18q','ask1Top19q','ask1Top20q','ask1Top21q','ask1Top22q','ask1Top23q',
            'ask1Top24q','ask1Top25q','ask1Top26q','ask1Top27q','ask1Top28q','ask1Top29q','ask1Top30q','ask1Top31q','ask1Top32q','ask1Top33q',
            'ask1Top34q','ask1Top35q','ask1Top36q','ask1Top37q','ask1Top38q','ask1Top39q','ask1Top40q','ask1Top41q','ask1Top42q','ask1Top43q',
            'ask1Top44q','ask1Top45q','ask1Top46q','ask1Top47q','ask1Top48q','ask1Top49q','ask1Top50q', "totalBidQuantity", "totalAskQuantity",
            "vwapBid", "vwapAsk"]]
    SZ.columns = ["sequenceNo", "skey", "time", "cum_volume", "cum_amount", "cum_trades_cnt", "prev_close",
                     "open", "high", "low", "close", 'bid10p', 'bid9p', 'bid8p', 'bid7p', 'bid6p', 'bid5p', 'bid4p',
                     'bid3p', 'bid2p', 'bid1p',
                     'ask1p', 'ask2p', 'ask3p', 'ask4p', 'ask5p', 'ask6p', 'ask7p', 'ask8p', 'ask9p', 'ask10p', 'bid10q',
                     'bid9q', 'bid8q',
                     'bid7q', 'bid6q', 'bid5q', 'bid4q', 'bid3q', 'bid2q', 'bid1q', 'ask1q', 'ask2q', 'ask3q', 'ask4q',
                     'ask5q', 'ask6q',
                     'ask7q', 'ask8q', 'ask9q', 'ask10q', 'bid10n', 'bid9n', 'bid8n', 'bid7n', 'bid6n', 'bid5n', 'bid4n',
                     'bid3n', 'bid2n', 'bid1n',
                     'ask1n', 'ask2n', 'ask3n', 'ask4n', 'ask5n', 'ask6n', 'ask7n', 'ask8n', 'ask9n', 'ask10n', 'bid1Top1q',
                     'bid1Top2q', 'bid1Top3q', 'bid1Top4q', 'bid1Top5q', 'bid1Top6q',
                     'bid1Top7q', 'bid1Top8q', 'bid1Top9q', 'bid1Top10q', 'bid1Top11q', 'bid1Top12q', 'bid1Top13q',
                     'bid1Top14q', 'bid1Top15q', 'bid1Top16q', 'bid1Top17q', 'bid1Top18q',
                     'bid1Top19q', 'bid1Top20q', 'bid1Top21q', 'bid1Top22q', 'bid1Top23q', 'bid1Top24q', 'bid1Top25q',
                     'bid1Top26q', 'bid1Top27q', 'bid1Top28q', 'bid1Top29q',
                     'bid1Top30q', 'bid1Top31q', 'bid1Top32q', 'bid1Top33q', 'bid1Top34q', 'bid1Top35q', 'bid1Top36q',
                     'bid1Top37q', 'bid1Top38q', 'bid1Top39q', 'bid1Top40q',
                     'bid1Top41q', 'bid1Top42q', 'bid1Top43q', 'bid1Top44q', 'bid1Top45q', 'bid1Top46q', 'bid1Top47q',
                     'bid1Top48q', 'bid1Top49q', 'bid1Top50q', 'ask1Top1q',
                     'ask1Top2q', 'ask1Top3q', 'ask1Top4q', 'ask1Top5q', 'ask1Top6q', 'ask1Top7q', 'ask1Top8q', 'ask1Top9q',
                     'ask1Top10q', 'ask1Top11q', 'ask1Top12q', 'ask1Top13q',
                     'ask1Top14q', 'ask1Top15q', 'ask1Top16q', 'ask1Top17q', 'ask1Top18q', 'ask1Top19q', 'ask1Top20q',
                     'ask1Top21q', 'ask1Top22q', 'ask1Top23q',
                     'ask1Top24q', 'ask1Top25q', 'ask1Top26q', 'ask1Top27q', 'ask1Top28q', 'ask1Top29q', 'ask1Top30q',
                     'ask1Top31q', 'ask1Top32q', 'ask1Top33q',
                     'ask1Top34q', 'ask1Top35q', 'ask1Top36q', 'ask1Top37q', 'ask1Top38q', 'ask1Top39q', 'ask1Top40q',
                     'ask1Top41q', 'ask1Top42q', 'ask1Top43q',
                     'ask1Top44q', 'ask1Top45q', 'ask1Top46q', 'ask1Top47q', 'ask1Top48q', 'ask1Top49q', 'ask1Top50q',
                     "total_bid_quantity", "total_ask_quantity", "total_bid_vwap", "total_ask_vwap"]
    SZ['date'] = int(y)
    SZ = SZ[SZ['skey'].isin(sl1)]
    SZ['time1'] = int(y) * 1000000000 + SZ['time']
    SZ["time"] = SZ['time'].astype('int64') * 1000
    SZ["clockAtArrival"] = SZ["time1"].astype(str).apply(
        lambda x: np.int64(datetime.datetime.strptime(x, '%Y%m%d%H%M%S%f').timestamp() * 1e6))
    SZ.drop("time1", axis=1, inplace=True)
    SZ['datetime'] = SZ["clockAtArrival"].apply(lambda x: datetime.datetime.fromtimestamp(x / 1e6))
    SZ = SZ.fillna(0)
    SZ["ordering"] = SZ.groupby("skey").cumcount() + 1

    for cols in ["total_bid_orders",'total_ask_orders','total_bid_levels', 'total_ask_levels', 'bid_trade_max_duration',
                 'ask_trade_max_duration', 'cum_canceled_buy_orders', 'cum_canceled_buy_volume', "cum_canceled_buy_amount",
                 "cum_canceled_sell_orders", 'cum_canceled_sell_volume',"cum_canceled_sell_amount", "has_missing"]:
        SZ[cols] = 0

    for col in ["skey", "date", "cum_trades_cnt", "total_bid_orders",
        'total_ask_orders', 'total_bid_levels', 'total_ask_levels', 'cum_canceled_buy_orders','cum_canceled_sell_orders',
            "ordering", 'bid_trade_max_duration', 'ask_trade_max_duration','has_missing', 'bid10n', 'bid9n', 'bid8n', 'bid7n', 'bid6n', 'bid5n', 'bid4n',
                     'bid3n', 'bid2n', 'bid1n','ask1n', 'ask2n', 'ask3n', 'ask4n', 'ask5n', 'ask6n', 'ask7n', 'ask8n', 'ask9n', 'ask10n',  'bid1Top1q',
                     'bid1Top2q', 'bid1Top3q', 'bid1Top4q', 'bid1Top5q', 'bid1Top6q',
                     'bid1Top7q', 'bid1Top8q', 'bid1Top9q', 'bid1Top10q', 'bid1Top11q', 'bid1Top12q', 'bid1Top13q',
                     'bid1Top14q', 'bid1Top15q', 'bid1Top16q', 'bid1Top17q', 'bid1Top18q',
                     'bid1Top19q', 'bid1Top20q', 'bid1Top21q', 'bid1Top22q', 'bid1Top23q', 'bid1Top24q', 'bid1Top25q',
                     'bid1Top26q', 'bid1Top27q', 'bid1Top28q', 'bid1Top29q',
                     'bid1Top30q', 'bid1Top31q', 'bid1Top32q', 'bid1Top33q', 'bid1Top34q', 'bid1Top35q', 'bid1Top36q',
                     'bid1Top37q', 'bid1Top38q', 'bid1Top39q', 'bid1Top40q',
                     'bid1Top41q', 'bid1Top42q', 'bid1Top43q', 'bid1Top44q', 'bid1Top45q', 'bid1Top46q', 'bid1Top47q',
                     'bid1Top48q', 'bid1Top49q', 'bid1Top50q', 'ask1Top1q',
                     'ask1Top2q', 'ask1Top3q', 'ask1Top4q', 'ask1Top5q', 'ask1Top6q', 'ask1Top7q', 'ask1Top8q', 'ask1Top9q',
                     'ask1Top10q', 'ask1Top11q', 'ask1Top12q', 'ask1Top13q',
                     'ask1Top14q', 'ask1Top15q', 'ask1Top16q', 'ask1Top17q', 'ask1Top18q', 'ask1Top19q', 'ask1Top20q',
                     'ask1Top21q', 'ask1Top22q', 'ask1Top23q',
                     'ask1Top24q', 'ask1Top25q', 'ask1Top26q', 'ask1Top27q', 'ask1Top28q', 'ask1Top29q', 'ask1Top30q',
                     'ask1Top31q', 'ask1Top32q', 'ask1Top33q',
                     'ask1Top34q', 'ask1Top35q', 'ask1Top36q', 'ask1Top37q', 'ask1Top38q', 'ask1Top39q', 'ask1Top40q',
                     'ask1Top41q', 'ask1Top42q', 'ask1Top43q',
                     'ask1Top44q', 'ask1Top45q', 'ask1Top46q', 'ask1Top47q', 'ask1Top48q', 'ask1Top49q', 'ask1Top50q']:
        SZ[col] = SZ[col].astype('int32')

    for col in ["cum_amount", "prev_close","high", "low", "close", 'open','bid10p','bid9p','bid8p','bid7p','bid6p','bid5p','bid4p','bid3p','bid2p','bid1p',
                          'ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p',
                          "total_bid_vwap", "total_ask_vwap"]:
        SZ[col] = (SZ[col]/10000).round(3)

    for cols in ["cum_canceled_sell_amount", "cum_canceled_buy_amount"]:
        SZ[cols] = SZ[cols].astype('float64')

    assert(sum(SZ[SZ["open"] != 0].groupby("skey")["open"].nunique() != 1) == 0)
    assert(sum(SZ[SZ["prev_close"] != 0].groupby("skey")["prev_close"].nunique() != 1) == 0)
    SZ["prev_close"] = np.where(SZ["time"] >= 91500000000, SZ.groupby("skey")["prev_close"].transform("max"), SZ["prev_close"]) 
    SZ["open"] = np.where(SZ["cum_volume"] > 0, SZ.groupby("skey")["open"].transform("max"), SZ["open"])
    assert(sum(SZ[SZ["open"] != 0].groupby("skey")["open"].nunique() != 1) == 0)
    assert(sum(SZ[SZ["prev_close"] != 0].groupby("skey")["prev_close"].nunique() != 1) == 0)
    assert(SZ[SZ["cum_volume"] > 0]["open"].min() > 0)

    startTm = datetime.datetime.now()
    SZ = SZ[["skey", "date", "time", "clockAtArrival", "datetime", "ordering", "has_missing", "cum_trades_cnt", "cum_volume", "cum_amount", "prev_close",
                            "open", "high", "low", "close", 'bid10p','bid9p','bid8p','bid7p','bid6p','bid5p','bid4p','bid3p','bid2p','bid1p',
                            'ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p', 'bid10q','bid9q','bid8q',
                             'bid7q','bid6q','bid5q','bid4q','bid3q','bid2q','bid1q', 'ask1q','ask2q','ask3q','ask4q','ask5q','ask6q',
                             'ask7q','ask8q','ask9q','ask10q', 'bid10n', 'bid9n', 'bid8n', 'bid7n', 'bid6n', 'bid5n', 'bid4n', 'bid3n', 'bid2n', 'bid1n', 
                             'ask1n', 'ask2n', 'ask3n', 'ask4n', 'ask5n', 'ask6n','ask7n', 'ask8n', 'ask9n', 'ask10n','bid1Top1q','bid1Top2q','bid1Top3q','bid1Top4q','bid1Top5q','bid1Top6q',
        'bid1Top7q','bid1Top8q','bid1Top9q','bid1Top10q','bid1Top11q','bid1Top12q','bid1Top13q','bid1Top14q','bid1Top15q','bid1Top16q','bid1Top17q','bid1Top18q',
        'bid1Top19q','bid1Top20q','bid1Top21q','bid1Top22q','bid1Top23q','bid1Top24q','bid1Top25q','bid1Top26q','bid1Top27q','bid1Top28q','bid1Top29q',
        'bid1Top30q','bid1Top31q','bid1Top32q','bid1Top33q','bid1Top34q','bid1Top35q','bid1Top36q','bid1Top37q','bid1Top38q','bid1Top39q','bid1Top40q',
        'bid1Top41q','bid1Top42q','bid1Top43q','bid1Top44q','bid1Top45q','bid1Top46q','bid1Top47q','bid1Top48q','bid1Top49q','bid1Top50q', 'ask1Top1q',
        'ask1Top2q','ask1Top3q','ask1Top4q','ask1Top5q','ask1Top6q','ask1Top7q','ask1Top8q','ask1Top9q','ask1Top10q','ask1Top11q','ask1Top12q','ask1Top13q',
        'ask1Top14q','ask1Top15q','ask1Top16q','ask1Top17q','ask1Top18q','ask1Top19q','ask1Top20q','ask1Top21q','ask1Top22q','ask1Top23q',
        'ask1Top24q','ask1Top25q','ask1Top26q','ask1Top27q','ask1Top28q','ask1Top29q','ask1Top30q','ask1Top31q','ask1Top32q','ask1Top33q',
        'ask1Top34q','ask1Top35q','ask1Top36q','ask1Top37q','ask1Top38q','ask1Top39q','ask1Top40q','ask1Top41q','ask1Top42q','ask1Top43q',
        'ask1Top44q','ask1Top45q','ask1Top46q','ask1Top47q','ask1Top48q','ask1Top49q','ask1Top50q',"total_bid_quantity", "total_ask_quantity","total_bid_vwap", "total_ask_vwap",
        "total_bid_orders",'total_ask_orders','total_bid_levels', 'total_ask_levels', 'bid_trade_max_duration', 'ask_trade_max_duration', 'cum_canceled_buy_orders', 'cum_canceled_buy_volume',
        "cum_canceled_buy_amount", "cum_canceled_sell_orders", 'cum_canceled_sell_volume',"cum_canceled_sell_amount"]]
        
    snapshot += [SZ]
    order += [orderLog]
    trade += [tradeLog]
    

    readPath = '\\\\192.168.10.34\\trading\\dailyRawData\\' + ii + '\\logs_***_zt_88_03_day_pcap\\mdTradePcap_SH_***.csv'
    dataPathLs = np.array(glob.glob(readPath))
    try:
        tradeLog = pd.read_csv(dataPathLs[0])
        print(dataPathLs[0])
    except:
        continue
    tradeLog = tradeLog.rename(columns={"time":'TransactTime', 'ID':'skey'})
    tradeLog = tradeLog[tradeLog['skey'].isin(sl2)]
    tradeLog['date'] = int(y)
    tradeLog['time1'] = int(y) * 1000000000 + tradeLog['TransactTime']
    tradeLog["TransactTime"] = tradeLog['TransactTime'].astype('int64') * 1000
    tradeLog["clockAtArrival"] = tradeLog["time1"].astype(str).apply(
        lambda x: np.int64(datetime.datetime.strptime(x, '%Y%m%d%H%M%S%f').timestamp() * 1e6))
    tradeLog.drop("time1", axis=1, inplace=True)
    tradeLog['datetime'] = tradeLog["clockAtArrival"].apply(lambda x: datetime.datetime.fromtimestamp(x / 1e6))
    tradeLog['TradePrice'] = (tradeLog['TradePrice'] / 10000).round(3)
    tradeLog = tradeLog.rename(columns={"TradeQty":"trade_qty", "TradePrice":"trade_price", 
                                        "TradeBSFlag":"trade_flag", "TradeAmount":"trade_money",
                                       "TradeIndex":"ApplSeqNum", "SellNo":"OfferApplSeqNum",
                                       "BuyNo":"BidApplSeqNum", 'TransactTime':"time"})
    tradeLog["trade_type"] = 1
    tradeLog["trade_flag"] = np.where(tradeLog["trade_flag"] == 'B', 1, np.where(
        tradeLog["trade_flag"] == 'S', 2, 0))
    for col in ["skey", "date", "ApplSeqNum", "BidApplSeqNum", "OfferApplSeqNum", "trade_qty", "trade_type", "trade_flag"]:
        tradeLog[col] = tradeLog[col].astype('int32')

    tradeLog = tradeLog[["skey", "date", "time", "clockAtArrival", "datetime", "ApplSeqNum", "trade_type", "trade_flag",
                                                 "trade_price", "trade_qty", "BidApplSeqNum", "OfferApplSeqNum"]]


    readPath = '\\\\192.168.10.34\\trading\\dailyRawData\\' + ii + '\\logs_***_zt_88_03_day_pcap\\mdL2Pcap_SH_***.csv'
    dataPathLs = np.array(glob.glob(readPath))
    try:
        SH = pd.read_csv(dataPathLs[0])
        print(dataPathLs[0])
    except:
        continue        
    SH = SH[["sequenceNo", "ID", "time", "cum_volume", "cum_amount", "cum_tradesCnt", "prevClose",
                            "open", "high", "low", "close", 'bid10p','bid9p','bid8p','bid7p','bid6p','bid5p','bid4p','bid3p','bid2p','bid1p',
                            'ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p', 'bid10q','bid9q','bid8q',
                              'bid7q','bid6q','bid5q','bid4q','bid3q','bid2q','bid1q', 'ask1q','ask2q','ask3q','ask4q','ask5q','ask6q',
                              'ask7q','ask8q','ask9q','ask10q', 'bid10n', 'bid9n', 'bid8n', 'bid7n', 'bid6n', 'bid5n', 'bid4n', 'bid3n', 'bid2n', 'bid1n', 
                              'ask1n', 'ask2n', 'ask3n', 'ask4n', 'ask5n', 'ask6n','ask7n', 'ask8n', 'ask9n', 'ask10n','bid1Top1q','bid1Top2q','bid1Top3q','bid1Top4q','bid1Top5q','bid1Top6q',
        'bid1Top7q','bid1Top8q','bid1Top9q','bid1Top10q','bid1Top11q','bid1Top12q','bid1Top13q','bid1Top14q','bid1Top15q','bid1Top16q','bid1Top17q','bid1Top18q',
        'bid1Top19q','bid1Top20q','bid1Top21q','bid1Top22q','bid1Top23q','bid1Top24q','bid1Top25q','bid1Top26q','bid1Top27q','bid1Top28q','bid1Top29q',
        'bid1Top30q','bid1Top31q','bid1Top32q','bid1Top33q','bid1Top34q','bid1Top35q','bid1Top36q','bid1Top37q','bid1Top38q','bid1Top39q','bid1Top40q',
        'bid1Top41q','bid1Top42q','bid1Top43q','bid1Top44q','bid1Top45q','bid1Top46q','bid1Top47q','bid1Top48q','bid1Top49q','bid1Top50q', 'ask1Top1q',
        'ask1Top2q','ask1Top3q','ask1Top4q','ask1Top5q','ask1Top6q','ask1Top7q','ask1Top8q','ask1Top9q','ask1Top10q','ask1Top11q','ask1Top12q','ask1Top13q',
        'ask1Top14q','ask1Top15q','ask1Top16q','ask1Top17q','ask1Top18q','ask1Top19q','ask1Top20q','ask1Top21q','ask1Top22q','ask1Top23q',
        'ask1Top24q','ask1Top25q','ask1Top26q','ask1Top27q','ask1Top28q','ask1Top29q','ask1Top30q','ask1Top31q','ask1Top32q','ask1Top33q',
        'ask1Top34q','ask1Top35q','ask1Top36q','ask1Top37q','ask1Top38q','ask1Top39q','ask1Top40q','ask1Top41q','ask1Top42q','ask1Top43q',
        'ask1Top44q','ask1Top45q','ask1Top46q','ask1Top47q','ask1Top48q','ask1Top49q','ask1Top50q', "totalBidQuantity", "totalAskQuantity",
        "vwapBid", "vwapAsk", "totalBidOrders", "totalAskOrders", "totalBidLevels", "totalAskLevels", "bidTradeMaxDuration", "askTradeMaxDuration", 
        "cum_canceledBuyOrders", "cum_canceledBuyVolume", "cum_canceledBuyAmount", "cum_canceledSellOrders", "cum_canceledSellVolume", "cum_canceledSellAmount"]]
    SH.columns = ["sequenceNo", "skey", "time", "cum_volume", "cum_amount", "cum_trades_cnt", "prev_close",
              "open", "high", "low", "close", 'bid10p', 'bid9p', 'bid8p', 'bid7p', 'bid6p', 'bid5p', 'bid4p',
              'bid3p', 'bid2p', 'bid1p',
              'ask1p', 'ask2p', 'ask3p', 'ask4p', 'ask5p', 'ask6p', 'ask7p', 'ask8p', 'ask9p', 'ask10p', 'bid10q',
              'bid9q', 'bid8q',
              'bid7q', 'bid6q', 'bid5q', 'bid4q', 'bid3q', 'bid2q', 'bid1q', 'ask1q', 'ask2q', 'ask3q', 'ask4q',
              'ask5q', 'ask6q',
              'ask7q', 'ask8q', 'ask9q', 'ask10q', 'bid10n', 'bid9n', 'bid8n', 'bid7n', 'bid6n', 'bid5n', 'bid4n',
              'bid3n', 'bid2n', 'bid1n',
              'ask1n', 'ask2n', 'ask3n', 'ask4n', 'ask5n', 'ask6n', 'ask7n', 'ask8n', 'ask9n', 'ask10n', 'bid1Top1q',
              'bid1Top2q', 'bid1Top3q', 'bid1Top4q', 'bid1Top5q', 'bid1Top6q',
              'bid1Top7q', 'bid1Top8q', 'bid1Top9q', 'bid1Top10q', 'bid1Top11q', 'bid1Top12q', 'bid1Top13q',
              'bid1Top14q', 'bid1Top15q', 'bid1Top16q', 'bid1Top17q', 'bid1Top18q',
              'bid1Top19q', 'bid1Top20q', 'bid1Top21q', 'bid1Top22q', 'bid1Top23q', 'bid1Top24q', 'bid1Top25q',
              'bid1Top26q', 'bid1Top27q', 'bid1Top28q', 'bid1Top29q',
              'bid1Top30q', 'bid1Top31q', 'bid1Top32q', 'bid1Top33q', 'bid1Top34q', 'bid1Top35q', 'bid1Top36q',
              'bid1Top37q', 'bid1Top38q', 'bid1Top39q', 'bid1Top40q',
              'bid1Top41q', 'bid1Top42q', 'bid1Top43q', 'bid1Top44q', 'bid1Top45q', 'bid1Top46q', 'bid1Top47q',
              'bid1Top48q', 'bid1Top49q', 'bid1Top50q', 'ask1Top1q',
              'ask1Top2q', 'ask1Top3q', 'ask1Top4q', 'ask1Top5q', 'ask1Top6q', 'ask1Top7q', 'ask1Top8q', 'ask1Top9q',
              'ask1Top10q', 'ask1Top11q', 'ask1Top12q', 'ask1Top13q',
              'ask1Top14q', 'ask1Top15q', 'ask1Top16q', 'ask1Top17q', 'ask1Top18q', 'ask1Top19q', 'ask1Top20q',
              'ask1Top21q', 'ask1Top22q', 'ask1Top23q',
              'ask1Top24q', 'ask1Top25q', 'ask1Top26q', 'ask1Top27q', 'ask1Top28q', 'ask1Top29q', 'ask1Top30q',
              'ask1Top31q', 'ask1Top32q', 'ask1Top33q',
              'ask1Top34q', 'ask1Top35q', 'ask1Top36q', 'ask1Top37q', 'ask1Top38q', 'ask1Top39q', 'ask1Top40q',
              'ask1Top41q', 'ask1Top42q', 'ask1Top43q',
              'ask1Top44q', 'ask1Top45q', 'ask1Top46q', 'ask1Top47q', 'ask1Top48q', 'ask1Top49q', 'ask1Top50q',
              "total_bid_quantity", "total_ask_quantity",
              "total_bid_vwap", "total_ask_vwap", "total_bid_orders", "total_ask_orders", "total_bid_levels", "total_ask_levels",
              "bid_trade_max_duration", "ask_trade_max_duration",
              "cum_canceled_buy_orders", "cum_canceled_buy_volume", "cum_canceled_buy_amount", "cum_canceled_sell_orders",
              "cum_canceled_sell_volume", "cum_canceled_sell_amount"]
    SH = SH[SH['skey'].isin(sl2)]
    SH['date'] = int(y)
    SH['time1'] = int(y) * 1000000000 + SH['time']
    SH["time"] = SH['time'].astype('int64') * 1000
    SH["clockAtArrival"] = SH["time1"].astype(str).apply(
        lambda x: np.int64(datetime.datetime.strptime(x, '%Y%m%d%H%M%S%f').timestamp() * 1e6))
    SH.drop("time1", axis=1, inplace=True)
    SH['datetime'] = SH["clockAtArrival"].apply(lambda x: datetime.datetime.fromtimestamp(x / 1e6))
    SH = SH.fillna(0)
    SH["ordering"] = SH.groupby("skey").cumcount() + 1

    SH["has_missing"] = 0
    for col in ["skey", "date", "cum_trades_cnt", "total_bid_orders",
                'total_ask_orders', 'total_bid_levels', 'total_ask_levels', 'cum_canceled_buy_orders',
                'cum_canceled_sell_orders',
                "ordering", 'bid_trade_max_duration', 'ask_trade_max_duration', 'has_missing', 'bid10n', 'bid9n', 'bid8n', 'bid7n', 'bid6n', 'bid5n', 'bid4n',
                      'bid3n', 'bid2n', 'bid1n','ask1n', 'ask2n', 'ask3n', 'ask4n', 'ask5n', 'ask6n', 'ask7n', 'ask8n', 'ask9n', 'ask10n',  'bid1Top1q',
                      'bid1Top2q', 'bid1Top3q', 'bid1Top4q', 'bid1Top5q', 'bid1Top6q',
                      'bid1Top7q', 'bid1Top8q', 'bid1Top9q', 'bid1Top10q', 'bid1Top11q', 'bid1Top12q', 'bid1Top13q',
                      'bid1Top14q', 'bid1Top15q', 'bid1Top16q', 'bid1Top17q', 'bid1Top18q',
                      'bid1Top19q', 'bid1Top20q', 'bid1Top21q', 'bid1Top22q', 'bid1Top23q', 'bid1Top24q', 'bid1Top25q',
                      'bid1Top26q', 'bid1Top27q', 'bid1Top28q', 'bid1Top29q',
                      'bid1Top30q', 'bid1Top31q', 'bid1Top32q', 'bid1Top33q', 'bid1Top34q', 'bid1Top35q', 'bid1Top36q',
                      'bid1Top37q', 'bid1Top38q', 'bid1Top39q', 'bid1Top40q',
                      'bid1Top41q', 'bid1Top42q', 'bid1Top43q', 'bid1Top44q', 'bid1Top45q', 'bid1Top46q', 'bid1Top47q',
                      'bid1Top48q', 'bid1Top49q', 'bid1Top50q', 'ask1Top1q',
                      'ask1Top2q', 'ask1Top3q', 'ask1Top4q', 'ask1Top5q', 'ask1Top6q', 'ask1Top7q', 'ask1Top8q', 'ask1Top9q',
                      'ask1Top10q', 'ask1Top11q', 'ask1Top12q', 'ask1Top13q',
                      'ask1Top14q', 'ask1Top15q', 'ask1Top16q', 'ask1Top17q', 'ask1Top18q', 'ask1Top19q', 'ask1Top20q',
                      'ask1Top21q', 'ask1Top22q', 'ask1Top23q',
                      'ask1Top24q', 'ask1Top25q', 'ask1Top26q', 'ask1Top27q', 'ask1Top28q', 'ask1Top29q', 'ask1Top30q',
                      'ask1Top31q', 'ask1Top32q', 'ask1Top33q',
                      'ask1Top34q', 'ask1Top35q', 'ask1Top36q', 'ask1Top37q', 'ask1Top38q', 'ask1Top39q', 'ask1Top40q',
                      'ask1Top41q', 'ask1Top42q', 'ask1Top43q',
                      'ask1Top44q', 'ask1Top45q', 'ask1Top46q', 'ask1Top47q', 'ask1Top48q', 'ask1Top49q', 'ask1Top50q']:
        SH[col] = SH[col].astype('int32')

    for cols in ["cum_amount", "prev_close","high", "low", "close", 'open','bid10p','bid9p','bid8p','bid7p','bid6p','bid5p','bid4p','bid3p','bid2p','bid1p',
                          'ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p', "cum_canceled_buy_amount", "cum_canceled_sell_amount"]:
        SH[cols] = (SH[cols]/10000).round(3)

    for cols in ['total_bid_vwap', "total_ask_vwap"]:
        SH[cols] = (SH[cols]/10000).round(3)

    assert (sum(SH[SH["open"] != 0].groupby("skey")["open"].nunique() != 1) == 0)
    assert (sum(SH[SH["prev_close"] != 0].groupby("skey")["prev_close"].nunique() != 1) == 0)
    SH["prev_close"] = np.where(SH["time"] >= 91500000000, SH.groupby("skey")["prev_close"].transform("max"),
                                SH["prev_close"])
    SH["open"] = np.where(SH["cum_volume"] > 0, SH.groupby("skey")["open"].transform("max"), SH["open"])
    assert (sum(SH[SH["open"] != 0].groupby("skey")["open"].nunique() != 1) == 0)
    assert (sum(SH[SH["prev_close"] != 0].groupby("skey")["prev_close"].nunique() != 1) == 0)
    assert (SH[SH["cum_volume"] > 0]["open"].min() > 0)

    SH = SH[
        ["skey", "date", "time", "clockAtArrival", "datetime", "ordering", "has_missing", "cum_trades_cnt", "cum_volume",
          "cum_amount", "prev_close",
          "open", "high", "low", "close", 'bid10p', 'bid9p', 'bid8p', 'bid7p', 'bid6p', 'bid5p', 'bid4p', 'bid3p', 'bid2p',
          'bid1p',
          'ask1p', 'ask2p', 'ask3p', 'ask4p', 'ask5p', 'ask6p', 'ask7p', 'ask8p', 'ask9p', 'ask10p', 'bid10q', 'bid9q',
          'bid8q',
          'bid7q', 'bid6q', 'bid5q', 'bid4q', 'bid3q', 'bid2q', 'bid1q', 'ask1q', 'ask2q', 'ask3q', 'ask4q', 'ask5q',
          'ask6q',
          'ask7q', 'ask8q', 'ask9q', 'ask10q', 'bid10n', 'bid9n', 'bid8n', 'bid7n', 'bid6n', 'bid5n', 'bid4n', 'bid3n',
          'bid2n', 'bid1n',
          'ask1n', 'ask2n', 'ask3n', 'ask4n', 'ask5n', 'ask6n', 'ask7n', 'ask8n', 'ask9n', 'ask10n', 'bid1Top1q',
          'bid1Top2q', 'bid1Top3q', 'bid1Top4q', 'bid1Top5q', 'bid1Top6q',
          'bid1Top7q', 'bid1Top8q', 'bid1Top9q', 'bid1Top10q', 'bid1Top11q', 'bid1Top12q', 'bid1Top13q', 'bid1Top14q',
          'bid1Top15q', 'bid1Top16q', 'bid1Top17q', 'bid1Top18q',
          'bid1Top19q', 'bid1Top20q', 'bid1Top21q', 'bid1Top22q', 'bid1Top23q', 'bid1Top24q', 'bid1Top25q', 'bid1Top26q',
          'bid1Top27q', 'bid1Top28q', 'bid1Top29q',
          'bid1Top30q', 'bid1Top31q', 'bid1Top32q', 'bid1Top33q', 'bid1Top34q', 'bid1Top35q', 'bid1Top36q', 'bid1Top37q',
          'bid1Top38q', 'bid1Top39q', 'bid1Top40q',
          'bid1Top41q', 'bid1Top42q', 'bid1Top43q', 'bid1Top44q', 'bid1Top45q', 'bid1Top46q', 'bid1Top47q', 'bid1Top48q',
          'bid1Top49q', 'bid1Top50q', 'ask1Top1q',
          'ask1Top2q', 'ask1Top3q', 'ask1Top4q', 'ask1Top5q', 'ask1Top6q', 'ask1Top7q', 'ask1Top8q', 'ask1Top9q',
          'ask1Top10q', 'ask1Top11q', 'ask1Top12q', 'ask1Top13q',
          'ask1Top14q', 'ask1Top15q', 'ask1Top16q', 'ask1Top17q', 'ask1Top18q', 'ask1Top19q', 'ask1Top20q', 'ask1Top21q',
          'ask1Top22q', 'ask1Top23q',
          'ask1Top24q', 'ask1Top25q', 'ask1Top26q', 'ask1Top27q', 'ask1Top28q', 'ask1Top29q', 'ask1Top30q', 'ask1Top31q',
          'ask1Top32q', 'ask1Top33q',
          'ask1Top34q', 'ask1Top35q', 'ask1Top36q', 'ask1Top37q', 'ask1Top38q', 'ask1Top39q', 'ask1Top40q', 'ask1Top41q',
          'ask1Top42q', 'ask1Top43q',
          'ask1Top44q', 'ask1Top45q', 'ask1Top46q', 'ask1Top47q', 'ask1Top48q', 'ask1Top49q', 'ask1Top50q',
          "total_bid_quantity", "total_ask_quantity", "total_bid_vwap", "total_ask_vwap",
          "total_bid_orders", 'total_ask_orders', 'total_bid_levels', 'total_ask_levels', 'bid_trade_max_duration',
          'ask_trade_max_duration', 'cum_canceled_buy_orders', 'cum_canceled_buy_volume',
          "cum_canceled_buy_amount", "cum_canceled_sell_orders", 'cum_canceled_sell_volume', "cum_canceled_sell_amount"]]

    snapshot += [SH]
    trade += [tradeLog]
    
    snapshot = pd.concat(snapshot).reset_index(drop=True)
    trade = pd.concat(trade).reset_index(drop=True)
    order = pd.concat(order).reset_index(drop=True)
    pd.to_pickle(snapshot, 'G:\\convertible_bond1\\' + ii + '\\snapshot.pkl')
    pd.to_pickle(order, 'G:\\convertible_bond1\\' + ii + '\\order.pkl')
    pd.to_pickle(trade, 'G:\\convertible_bond1\\' + ii + '\\trade.pkl')
    print(ii + ' done')


C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


\\192.168.10.34\trading\dailyRawData\20201014\logs_20201014_zs_96_03_day_pcap\mdOrderPcap_SZ_20201014_0900.csv
\\192.168.10.34\trading\dailyRawData\20201014\logs_20201014_zs_96_03_day_pcap\mdTradePcap_SZ_20201014_0900.csv
\\192.168.10.34\trading\dailyRawData\20201014\logs_20201014_zs_96_03_day_pcap\mdL2Pcap_SZ_20201014_0900.csv
\\192.168.10.34\trading\dailyRawData\20201014\logs_20201014_zt_88_03_day_pcap\mdTradePcap_SH_20201014_0900.csv
\\192.168.10.34\trading\dailyRawData\20201014\logs_20201014_zt_88_03_day_pcap\mdL2Pcap_SH_20201014_0900.csv
20201014 done


In [1]:
import os
import glob
import datetime
import numpy as np
import pandas as pd

pd.set_option('max_rows', 100)
pd.set_option('max_columns', 100)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

startDate = '20201123'
endDate = '20201123'

sl = ['SZ123015', 'SZ123032', 'SZ128070', 'SZ123042', 'SZ123034',
       'SZ123048', 'SZ123061', 'SH113576', 'SZ123065', 'SZ123030',
       'SZ123045', 'SZ123013', 'SZ127004', 'SZ128103', 'SZ128073',
       'SZ128041', 'SZ123012', 'SZ123062', 'SH110044', 'SZ128079',
       'SZ123052', 'SZ128036', 'SZ123014', 'SZ128091', 'SZ123018',
       'SZ123031', 'SZ123038', 'SH113035', 'SZ128052', 'SZ128053',
       'SZ128039', 'SZ128050', 'SZ123023', 'SH113555', 'SZ127008',
       'SZ123047', 'SZ123051', 'SZ123024', 'SZ123029', 'SZ128076',
       'SZ123026', 'SZ123027', 'SZ128112', 'SH113521', 'SZ123057',
       'SZ128108', 'SH113520', 'SZ123046', 'SH113038', 'SH113011']
sl1 = [i[2:] for i in sl if i[:2] == 'SZ']
sl2 = [i[2:] for i in sl if i[:2] == 'SH']
sl1 = [int(i) + 2000000 for i in sl1]
sl2 = [int(i) + 1000000 for i in sl2]

readPath = '\\\\192.168.10.34\\trading\\dailyRawData\\***'
dataPathLs = np.array(glob.glob(readPath))
dateLs = np.array([os.path.basename(i) for i in dataPathLs])
dateLs = dateLs[(dateLs >= startDate) & (dateLs <= endDate)]

for ii in dateLs:
    order = []
    trade = []
    snapshot = []
    os.mkdir('G:\\convertible_bond1\\' + ii)
    y = ii

    readPath = '\\\\192.168.10.34\\trading\\dailyRawData\\' + ii + '\\logs_***_zs_96_03_day_pcap\\mdTradePcap_SZ_***.csv'
    dataPathLs = np.array(glob.glob(readPath))
    if len(dataPathLs) == 0:
        readPath = '\\\\192.168.10.34\\trading\\dailyRawData\\' + ii + '\\logs_***_zs_88_04_day_pcap\\mdTradePcap_SZ_***.csv'
        dataPathLs = np.array(glob.glob(readPath))
        assert(len(dataPathLs) == 1)
    try:
        tradeLog = pd.read_csv(dataPathLs[0])
        print(dataPathLs[0])
    except:
        continue
    
    tradeLog = tradeLog.rename(columns={"time":'TransactTime', 'ID':'skey'})
    tradeLog["TradeBSFlag"] = 'N'
    tradeLog = tradeLog[tradeLog['skey'].isin(sl1)]
    tradeLog['date'] = int(y)
    tradeLog['time1'] = int(y) * 1000000000 + tradeLog['TransactTime']
    tradeLog["TransactTime"] = tradeLog['TransactTime'].astype('int64') * 1000
    tradeLog["clockAtArrival"] = tradeLog["time1"].astype(str).apply(
        lambda x: np.int64(datetime.datetime.strptime(x, '%Y%m%d%H%M%S%f').timestamp() * 1e6))
    tradeLog.drop("time1", axis=1, inplace=True)
    tradeLog['datetime'] = tradeLog["clockAtArrival"].apply(lambda x: datetime.datetime.fromtimestamp(x / 1e6))
    tradeLog['TradePrice'] = (tradeLog['TradePrice'] / 10000).round(3)
    tradeLog = tradeLog.rename(columns={"TradeQty":"trade_qty", "TradePrice":"trade_price", "ExecType":"trade_type", 'TransactTime':'time'})
    tradeLog["trade_flag"] = 0
    tradeLog["trade_type"] = np.where(tradeLog["trade_type"] == 'F', 1, tradeLog["trade_type"])
    for col in ["skey", "date", "ApplSeqNum", "BidApplSeqNum", "OfferApplSeqNum", "trade_qty", "trade_type", "trade_flag"]:
        tradeLog[col] = tradeLog[col].astype('int32')

    tradeLog = tradeLog[["skey", "date", "time", "clockAtArrival", "datetime", "ApplSeqNum", "trade_type", "trade_flag",
                                                 "trade_price", "trade_qty", "BidApplSeqNum", "OfferApplSeqNum"]]
    
    trade += [tradeLog]
    

    readPath = '\\\\192.168.10.34\\trading\\dailyRawData\\' + ii + '\\logs_***_zt_88_06_day_pcap\\mdTradePcap_SH_***.csv'
    dataPathLs = np.array(glob.glob(readPath))
    if len(dataPathLs) == 0:
        readPath = '\\\\192.168.10.34\\trading\\dailyRawData\\' + ii + '\\logs_***_zt_52_03_day_pcap\\mdTradePcap_SH_***.csv'
        dataPathLs = np.array(glob.glob(readPath))
        assert(len(dataPathLs) == 1)
    try:
        tradeLog = pd.read_csv(dataPathLs[0])
        print(dataPathLs[0])
    except:
        continue
    tradeLog = tradeLog.rename(columns={"time":'TransactTime', 'ID':'skey'})
    tradeLog = tradeLog[tradeLog['skey'].isin(sl2)]
    tradeLog['date'] = int(y)
    tradeLog['time1'] = int(y) * 1000000000 + tradeLog['TransactTime']
    tradeLog["TransactTime"] = tradeLog['TransactTime'].astype('int64') * 1000
    tradeLog["clockAtArrival"] = tradeLog["time1"].astype(str).apply(
        lambda x: np.int64(datetime.datetime.strptime(x, '%Y%m%d%H%M%S%f').timestamp() * 1e6))
    tradeLog.drop("time1", axis=1, inplace=True)
    tradeLog['datetime'] = tradeLog["clockAtArrival"].apply(lambda x: datetime.datetime.fromtimestamp(x / 1e6))
    tradeLog['TradePrice'] = (tradeLog['TradePrice'] / 10000).round(3)
    tradeLog = tradeLog.rename(columns={"TradeQty":"trade_qty", "TradePrice":"trade_price", 
                                        "TradeBSFlag":"trade_flag", "TradeAmount":"trade_money",
                                       "TradeIndex":"ApplSeqNum", "SellNo":"OfferApplSeqNum",
                                       "BuyNo":"BidApplSeqNum", 'TransactTime':"time"})
    tradeLog["trade_type"] = 1
    tradeLog["trade_flag"] = np.where(tradeLog["trade_flag"] == 'B', 1, np.where(
        tradeLog["trade_flag"] == 'S', 2, 0))
    for col in ["skey", "date", "ApplSeqNum", "BidApplSeqNum", "OfferApplSeqNum", "trade_qty", "trade_type", "trade_flag"]:
        tradeLog[col] = tradeLog[col].astype('int32')

    tradeLog = tradeLog[["skey", "date", "time", "clockAtArrival", "datetime", "ApplSeqNum", "trade_type", "trade_flag",
                                                 "trade_price", "trade_qty", "BidApplSeqNum", "OfferApplSeqNum"]]
    
    trade += [tradeLog]
    trade = pd.concat(trade).reset_index(drop=True)
    pd.to_pickle(trade, 'G:\\convertible_bond1\\' + ii + '\\trade.pkl')
    print(ii + ' done')


\\192.168.10.34\trading\dailyRawData\20201123\logs_20201123_zs_96_03_day_pcap\mdTradePcap_SZ_20201123_0900.csv
\\192.168.10.34\trading\dailyRawData\20201123\logs_20201123_zt_88_06_day_pcap\mdTradePcap_SH_20201123_0900.csv
20201123 done


In [15]:
import pandas as pd
import numpy as np
import glob
y = '20201014'
sl1 = [123061, 128091]
sl2 = [2123061, 2128091]
for i in [y]:
    p1 = '\\\\192.168.10.34\\trading\\dailyRawData\\' + i + '\\***\mdTradePcap_SZ_***.csv'
    p1 = np.array(glob.glob(p1))
    if len(p1) == 2:
        print(p1[0])
        data1 = pd.read_csv(p1[0])
        print(p1[1])
        data2 = pd.read_csv(p1[1])
    else:
        print(p1[0])
        data1 = pd.read_csv(p1[0])
    
    data3 = []
    for stock in [123061, 128091]:
        p2 = '\\\\192.168.10.34\\random_backup\\Kevin_zhenyu\\KR_daily_data\\' + i + '\\SZ\\tick\\' + str(stock) + '.csv'
        p2 = pd.read_csv(p2)
        p2['SecurityID'] = stock
        data3 += [p2]
    data3 = pd.concat(data3)

In [17]:
data3[data3['ExecType'] == 'F'].groupby('SecurityID')['Qty'].sum()

SecurityID
123061    5572165
128091     362180
Name: Qty, dtype: int64

In [19]:
data1[(data1['ID'].isin(sl2)) & (data1['ExecType'] == 'F')].groupby(['ID'])['TradeQty'].sum()

ID
2123061    5572165
2128091     362180
Name: TradeQty, dtype: int64

In [20]:
data2[(data2['ID'].isin(sl2)) & (data2['ExecType'] == 'F')].groupby(['ID'])['TradeQty'].sum()

ID
2123061    5572065
2128091     362140
Name: TradeQty, dtype: int64

In [27]:
# SZ version
import datetime
y = '20201014'
sl1 = [123061, 128091]
sl2 = [2123061, 2128091]

od = pd.read_pickle('A:\\convertible_bond\\' + y + '\\trade.pkl')
od = od[~od['skey'].isin(sl2)]
# data1/data2
tradeLog = data1[data1['ID'].isin(sl2)]
tradeLog = tradeLog.rename(columns={"time":'TransactTime', 'ID':'skey'})
tradeLog["TradeBSFlag"] = 'N'
tradeLog = tradeLog[tradeLog['skey'].isin(sl2)]
tradeLog['date'] = int(y)
tradeLog['time1'] = int(y) * 1000000000 + tradeLog['TransactTime']
tradeLog["TransactTime"] = tradeLog['TransactTime'].astype('int64') * 1000
tradeLog["clockAtArrival"] = tradeLog["time1"].astype(str).apply(
    lambda x: np.int64(datetime.datetime.strptime(x, '%Y%m%d%H%M%S%f').timestamp() * 1e6))
tradeLog.drop("time1", axis=1, inplace=True)
tradeLog['datetime'] = tradeLog["clockAtArrival"].apply(lambda x: datetime.datetime.fromtimestamp(x / 1e6))
tradeLog['TradePrice'] = (tradeLog['TradePrice'] / 10000).round(3)
tradeLog = tradeLog.rename(columns={"TradeQty":"trade_qty", "TradePrice":"trade_price", "ExecType":"trade_type", 'TransactTime':'time'})
tradeLog["trade_flag"] = 0
tradeLog["trade_type"] = np.where(tradeLog["trade_type"] == 'F', 1, tradeLog["trade_type"])
for col in ["skey", "date", "ApplSeqNum", "BidApplSeqNum", "OfferApplSeqNum", "trade_qty", "trade_type", "trade_flag"]:
    tradeLog[col] = tradeLog[col].astype('int32')

tradeLog = tradeLog[["skey", "date", "time", "clockAtArrival", "datetime", "ApplSeqNum", "trade_type", "trade_flag",
                                             "trade_price", "trade_qty", "BidApplSeqNum", "OfferApplSeqNum"]]
od = pd.concat([od, tradeLog]).reset_index(drop=True)
od.to_pickle(r'A:\convertible_bond\20201014\trade.pkl')
    
# data3

In [34]:
od[(od['skey'].isin(sl2)) & (od['trade_type'] == 1)].groupby(['skey'])['trade_qty'].sum()

skey
2123061    5572165
2128091     362180
Name: trade_qty, dtype: int32

In [48]:
import pandas as pd
import numpy as np
import glob
y = '20201020'
sl1 = [110044, 113011, 113035, 113038, 113520, 113521, 113555, 113576]
sl2 = [1110044, 1113011, 1113035, 1113038, 1113520, 1113521, 1113555, 1113576]
for i in [y]:
    p1 = '\\\\192.168.10.34\\trading\\dailyRawData\\' + i + '\\***\mdTradePcap_SH_***.csv'
    p1 = np.array(glob.glob(p1))
    if len(p1) == 2:
        print(p1[0])
        data1 = pd.read_csv(p1[0])
        print(p1[1])
        data2 = pd.read_csv(p1[1])
    else:
        print(p1[0])
        data1 = pd.read_csv(p1[0])
    
    data3 = []
    for stock in sl1:
        p2 = '\\\\192.168.10.34\\random_backup\\Kevin_zhenyu\\KR_daily_data\\' + i + '\\SH\\tick\\' + str(stock) + '.csv'
        p2 = pd.read_csv(p2)
        p2['SecurityID'] = stock
        data3 += [p2]
    data3 = pd.concat(data3)

\\192.168.10.34\trading\dailyRawData\20201020\logs_20201020_zt_52_03_day_pcap\mdTradePcap_SH_20201020_0900.csv
\\192.168.10.34\trading\dailyRawData\20201020\logs_20201020_zt_88_03_day_pcap\mdTradePcap_SH_20201020_0900.csv


In [49]:
data3.groupby('SecurityID')['TradeQty'].sum()

SecurityID
110044     974821
113011      80840
113035     235218
113038     241803
113520     214877
113521     303408
113555     121736
113576    1126507
Name: TradeQty, dtype: int64

In [45]:
od[(od['skey'].isin(sl2)) & (od['trade_type'] == 1)].groupby(['skey'])['trade_qty'].sum()

skey
1110044    130608
1113011    351370
1113035    294535
1113038    106609
1113520     71256
1113521     50113
1113555    220319
1113576    498459
Name: trade_qty, dtype: int32

In [50]:
data1[(data1['ID'].isin(sl2))].groupby(['ID'])['TradeQty'].sum()

ID
1110044     974821
1113011      80840
1113035     235218
1113038     241803
1113520     214877
1113521     303408
1113555     121736
1113576    1126507
Name: TradeQty, dtype: int64

In [51]:
data2[(data2['ID'].isin(sl2))].groupby(['ID'])['TradeQty'].sum()

ID
1110044    321972
1113011     18961
1113035     57874
1113038     54469
1113520     66004
1113521     24576
1113555     30177
1113576    186820
Name: TradeQty, dtype: int64

In [44]:
# SH version
import datetime
y = '20201020'
sl1 = [110044, 113011, 113035, 113038, 113520, 113521, 113555, 113576]
sl2 = [1110044, 1113011, 1113035, 1113038, 1113520, 1113521, 1113555, 1113576]

od = pd.read_pickle('A:\\convertible_bond\\' + y + '\\trade.pkl')
od = od[~od['skey'].isin(sl2)]
# data3
tradeLog = data3
tradeLog["date"] = tradeLog["TradeTime"].iloc[0]//1000000000
tradeLog = tradeLog.rename(columns={"TradeQty":"trade_qty", "TradePrice":"trade_price", 
                                    "TradeBSFlag":"trade_flag", "TradeAmount":"trade_money",
                                   "TradeIndex":"ApplSeqNum", "SellNo":"OfferApplSeqNum",
                                   "BuyNo":"BidApplSeqNum"})
tradeLog["trade_type"] = 1
tradeLog["skey"] = tradeLog["SecurityID"] + 1000000
tradeLog["clockAtArrival"] = tradeLog["TradeTime"].astype(str).apply(lambda x: np.int64(datetime.datetime.strptime(x, '%Y%m%d%H%M%S%f').timestamp()*1e6))
tradeLog['datetime'] = tradeLog["clockAtArrival"].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
tradeLog["time"] = (tradeLog['TradeTime'] - int(tradeLog['TradeTime'].iloc[0]//1000000000*1000000000)).astype(np.int64)*1000
tradeLog["trade_flag"] = np.where(tradeLog["trade_flag"] == 'B', 1, np.where(
    tradeLog["trade_flag"] == 'S', 2, 0))
for col in ["skey", "date", "ApplSeqNum", "BidApplSeqNum", "OfferApplSeqNum", "trade_qty", "trade_type", "trade_flag"]:
    tradeLog[col] = tradeLog[col].astype('int32')

tradeLog = tradeLog[["skey", "date", "time", "clockAtArrival", "datetime", "ApplSeqNum", "trade_type", "trade_flag",
                                             "trade_price", "trade_qty", "BidApplSeqNum", "OfferApplSeqNum"]]
assert(tradeLog['skey'].nunique() == 8)
od = pd.concat([od, tradeLog]).reset_index(drop=True)
od.to_pickle(r'A:\convertible_bond\20201014\trade.pkl')
print(od['skey'].nunique())
    
# data3